## Faster opnorm for IntervalMatrix

https://github.com/JuliaReach/IntervalMatrices.jl/pull/125

In [27]:
using Revise, LazySets, IntervalMatrices, LinearAlgebra
using BenchmarkTools

In [135]:
#   26.181 μs (8 allocations: 234.64 KiB)
function opnorm1(A::IntervalMatrix, p::Real=Inf)
    if p == Inf || p == 1
        return LinearAlgebra.opnorm(max.(abs.(inf(A)), abs.(sup(A))), p)
    else
        error("the interval matrix norm for this value of p=$p is not implemented")
    end
end

opnorm1 (generic function with 2 methods)

In [136]:
#   26.080 μs (8 allocations: 234.64 KiB)
function opnorm2(A::IntervalMatrix, p::Real=Inf)
    if p == Inf || p == 1
        return LinearAlgebra.opnorm(max.((-).(inf(A)), sup(A)), p)
    else
        error("the interval matrix norm for this value of p=$p is not implemented")
    end
end

#   33.279 μs (10 allocations: 312.84 KiB)
function opnorm2b(A::IntervalMatrix, p::Real=Inf)
    if p == Inf || p == 1
        return LinearAlgebra.opnorm(max.(-inf(A), sup(A)), p)
    else
        error("the interval matrix norm for this value of p=$p is not implemented")
    end
end

opnorm2b (generic function with 2 methods)

In [137]:
function opnorm3(A::IntervalMatrix, p::Real=Inf)
    if p == Inf
        return _opnorm_inf_a(A)
    elseif p == 1
        return _opnorm_1(A)
    else
        error("the interval matrix norm for this value of p=$p is not implemented")
    end
end

function _opnorm_1(A) end

function _opnorm_inf_a(A::IntervalMatrix{N}) where {N}
    res = zero(N)
    m, n = size(A)
    @inbounds @simd for i in 1:m
        acc = zero(N)
        for j in 1:m
            x = A[i, j]
            acc += max(-inf(x), sup(x))
        end
        if acc > res
            res = acc
        end
    end
    return res
end

function _opnorm_inf_b(A::IntervalMatrix{N}) where {N}
    res = zero(N)
    m, n = size(A)
    @inbounds @simd for i in 1:m
        acc = zero(N)
        for j in 1:m
            x = A[i, j]
            acc += max(abs(inf(x)), abs(sup(x)))
        end
        if acc > res
            res = acc
        end
    end
    return res
end

function _opnorm_inf_c(A::IntervalMatrix{N}) where {N}
    m, n = size(A)
    acc = zeros(N, m)
    @inbounds @simd for j in 1:m
        for i in 1:n
            x = A[i, j]
            acc[i] += max(-inf(x), sup(x))
        end
    end
    return maximum(acc)
end

function _opnorm_inf_d(A::IntervalMatrix{N}) where {N}
    m, n = size(A)
    acc = zeros(N, m)
    @inbounds @simd for j in 1:m
        for i in 1:n
            x = A[i, j]
            acc[i] += max(abs(inf(x)), abs(sup(x)))
        end
    end
    return maximum(acc)
end

function _opnorm_inf_e(A::IntervalMatrix{N}) where {N}
    m, n = size(A)
    res = zero(N)
    At = transpose(A)
    @inbounds @simd for j in 1:m
        acc = zero(N)
        for i in 1:n
            x = At[i, j]
            acc += max(abs(inf(x)), abs(sup(x)))
        end
        if acc > res
            res = acc
        end
    end
    return res
end

function _opnorm_inf_f(A::IntervalMatrix{N}) where {N}
    m, n = size(A)
    res = zero(N)
    At = transpose(A)
    @inbounds @simd for j in 1:m
        acc = zero(N)
        for i in 1:n
            x = At[i, j]
            acc += max(-inf(x), sup(x))
        end
        if acc > res
            res = acc
        end
    end
    return res
end

#=
Results:
  48.442 μs (0 allocations: 0 bytes)
  18.663 μs (0 allocations: 0 bytes)
  49.184 μs (1 allocation: 896 bytes)
  20.629 μs (1 allocation: 896 bytes)
  18.683 μs (0 allocations: 0 bytes) # best is version e
  50.652 μs (0 allocations: 0 bytes)
=#

_opnorm_inf_f (generic function with 1 method)

In [138]:
A = rand(IntervalMatrix, 100);

In [139]:
opnorm1(A)

127.15752594236561

In [140]:
opnorm2(A)

127.15752594236561

In [141]:
opnorm2b(A)

127.15752594236561

In [142]:
@show _opnorm_inf_a(A)
@show _opnorm_inf_b(A)
@show _opnorm_inf_c(A)
@show _opnorm_inf_d(A)
@show _opnorm_inf_e(A)
@show _opnorm_inf_f(A)

_opnorm_inf_a(A) = 127.15752594236561
_opnorm_inf_b(A) = 127.15752594236561
_opnorm_inf_c(A) = 127.15752594236561
_opnorm_inf_d(A) = 127.15752594236561
_opnorm_inf_e(A) = 127.15752594236561
_opnorm_inf_f(A) = 127.15752594236561


127.15752594236561

In [143]:
@btime opnorm1($A)

  26.181 μs (8 allocations: 234.64 KiB)


127.15752594236561

In [144]:
@btime opnorm2($A)

  26.080 μs (8 allocations: 234.64 KiB)


127.15752594236561

In [145]:
@btime opnorm2b($A)

  33.279 μs (10 allocations: 312.84 KiB)


127.15752594236561

In [146]:
@btime _opnorm_inf_a($A)
@btime _opnorm_inf_b($A)
@btime _opnorm_inf_c($A)
@btime _opnorm_inf_d($A)
@btime _opnorm_inf_e($A) # fastest version
@btime _opnorm_inf_f($A)

  48.442 μs (0 allocations: 0 bytes)
  18.663 μs (0 allocations: 0 bytes)
  49.184 μs (1 allocation: 896 bytes)
  20.629 μs (1 allocation: 896 bytes)
  18.683 μs (0 allocations: 0 bytes)
  50.652 μs (0 allocations: 0 bytes)


127.15752594236561

Result: `max(|inf(i)|, |sup(i)|) == max(-inf(i), sup(i))` for interval `i`

In [113]:
f(x) = max(abs(inf(x)), abs(sup(x)))
g(x) = max(-inf(x), sup(x))

g (generic function with 1 method)

In [114]:
x = [rand(IntervalMatrices.Interval) for _ in 1:1000];

In [115]:
@btime f.($x);

  1.633 μs (1 allocation: 7.94 KiB)


In [116]:
@btime g.($x);

  1.019 μs (1 allocation: 7.94 KiB)


In [147]:
(26 - 18.6)/26 * 100

28.461538461538456

In [149]:
(26.181 - 26.080)/(26.181) * 100

0.38577594438716106